### Guide - Save a styled DataFrame as an image using Matplotlib and Selenium

#### Matplotlib: Render a table using Matplotlib and save it as a PNG file using Pandas Styler

#### Selenium: Capture a screenshot of the rendered table as an image


In [1]:
import pandas as pd
import plotly.io as pio
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
#!pip install selenium webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import display, HTML
import os

### Inputs

In [2]:
# Recreate the DataFrame similar to the visible table in your image
data = {
    '%-ile': ['20yr', '10yr', '7yr', '5yr', '5yr pre-Covid', 'pre-Covid', 'Recessionary Bear Markets', 'Non-Recessionary Bear Markets', 'Bull'],
    'z-score': [-0.9, -1.0, -0.8, -0.7, -1.5, -2.0, -1.3, -1.0, -0.7],
    'Max': ['61%', '61%', '55%', '55%', '52%', '50%', '44%', '42%', '41%'],
    'Min': ['1%', '1%', '2%', '2%', '2%', '2%', '3%', '4%', '2%'],
    'Median': ['19%', '18%', '15%', '14%', '12%', '10%', '11%', '10%', '12%'],
    'Mean': ['20%', '18%', '16%', '15%', '13%', '11%', '12%', '11%', '13%'],
    'Last Value': ['1.4%', '1.4%', '1.4%', '1.4%', '1.4%', '1.4%', '1.7%', '1.6%', '1.6%'],
    'Prob Payout': ['3.3%', '3.0%', '2.7%', '2.5%', '2.3%', '2.0%', '4.0%', '6.0%', '3.0%']
}

df_out = pd.DataFrame(data)
# Display the DataFrame
(df_out)

,%-ile,z-score,Max,Min,Median,Mean,Last Value,Prob Payout
0,20yr,-0.9,61%,1%,19%,20%,1.4%,3.3%
1,10yr,-1.0,61%,1%,18%,18%,1.4%,3.0%
2,7yr,-0.8,55%,2%,15%,16%,1.4%,2.7%
3,5yr,-0.7,55%,2%,14%,15%,1.4%,2.5%
4,5yr pre-Covid,-1.5,52%,2%,12%,13%,1.4%,2.3%
5,pre-Covid,-2.0,50%,2%,10%,11%,1.4%,2.0%
6,Recessionary Bear Markets,-1.3,44%,3%,11%,12%,1.7%,4.0%
7,Non-Recessionary Bear Markets,-1.0,42%,4%,10%,11%,1.6%,6.0%
8,Bull,-0.7,41%,2%,12%,13%,1.6%,3.0%


### Funtions

#### 1. style_df

In [3]:
# Function to style a DataFrame for better visualization
def style_df(df, caption, source_note):
    # Apply original styling to the table
    df_style = (
        df.style
        .background_gradient(subset=df.columns[1:], cmap='RdYlGn')  # Apply gradient to numeric columns except the first
        .format({col: "{:.1f}%" for col in df.columns[1:] if pd.api.types.is_numeric_dtype(df[col])})  # Format numeric columns
        .set_properties(**{'text-align': 'center'})  # Center align all columns
    )

    # Set table styles for better presentation
    styles = [
        {"selector": "th", "props": [("text-align", "center")]},  # Center align headers
        {"selector": "thead th:first-child", "props": [("text-align", "center")]},  # Align first column to the left
        {"selector": "tbody tr td:first-child", "props": [("width", "200px")]},  # Set width of first column
        {"selector": "tbody td", "props": [("border-right", "1px solid black")]}  # Add border to columns
    ]
    df_style = df_style.set_table_styles(styles)

    # Set table attributes
    df_style.set_table_attributes('style="width: 100%; border-collapse: collapse; border: 1px solid black;"')
    df_style.set_caption(caption)  # Add table caption

    # Add source note as a footnote
    source_html = f"<p style='font-size:10px; margin-top: 10px;'>Source: {source_note}</p>"

    # Convert the styled DataFrame to HTML and append the source note
    full_html = f"{df_style.to_html()}{source_html}"

    return full_html

#### 2. table_to_png

In [4]:
# Function to save a styled DataFrame with a heatmap as a PNG
def save_styled_df_as_png(df, output_file):
    # Add background gradient (heatmap) to numeric columns
    styled_html = df.style.background_gradient(cmap="coolwarm").to_html()

    # Save the HTML table to a temporary file
    html_file = os.path.abspath("temp_table.html")  # Get absolute path to avoid file resolution issues
    with open(html_file, "w") as f:
        f.write(styled_html)

    # Set up Selenium WebDriver in headless mode
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920x1080")

    # Use WebDriver Manager to handle ChromeDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Open the absolute HTML file path and take a screenshot
    driver.get(f"file://{html_file}")
    driver.save_screenshot(output_file)
    driver.quit()

    print(f"PNG saved as '{output_file}' in the current directory.")

### RUN

In [7]:
# Example DataFrame
data = {
    'Column1': [1, 2, 3],
    'Column2': [4, 5, 6],
    'Column3': [7, 8, 9]
}
df = pd.DataFrame(data)

In [9]:
# Save the DataFrame as a PNG
output_file = "styled_table_with_heatmap.png"
save_styled_df_as_png(df_out, output_file)

PNG saved as 'styled_table_with_heatmap.png' in the current directory.


## DEV

In [10]:
# Function to save a styled DataFrame with a heatmap as a PNG
def save_styled_df_as_png(df, output_file):
    # Add background gradient (heatmap) to numeric columns
    styled_html = df.style.background_gradient(cmap="coolwarm").set_table_attributes(
        'style="border-collapse: collapse; width: auto; border: 1px solid black;"').to_html()

    # Save the HTML table to a temporary file
    html_file = os.path.abspath("temp_table.html")  # Get absolute path to avoid file resolution issues
    with open(html_file, "w") as f:
        f.write(f"""
        <html>
        <head>
        <style>
        body {{
            margin: 0;
            padding: 0;
            display: inline-block;
        }}
        </style>
        </head>
        <body>
        {styled_html}
        </body>
        </html>
        """)

    # Set up Selenium WebDriver in headless mode
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--start-maximized")  # Ensure browser is maximized

    # Automatically detect and download the correct ChromeDriver version
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Open the temporary HTML file
    driver.get(f"file://{html_file}")

    # Dynamically adjust window size to match the table size
    table_width = driver.execute_script("return document.body.scrollWidth")
    table_height = driver.execute_script("return document.body.scrollHeight")
    driver.set_window_size(table_width, table_height)

    # Take a screenshot
    driver.save_screenshot(output_file)
    driver.quit()

    print(f"PNG saved as '{output_file}' in the current directory.")

    # Display the styled table in the notebook
    display(HTML(styled_html))

# Example DataFrame
data = {
    'Column1': [1, 2, 3],
    'Column2': [4, 5, 6],
    'Column3': [7, 8, 9]
}
df = pd.DataFrame(data)

# Save the DataFrame as a PNG
output_file = "styled_table_with_heatmap.png"
save_styled_df_as_png(df, output_file)


PNG saved as 'styled_table_with_heatmap.png' in the current directory.


,Column1,Column2,Column3
0,1,4,7
1,2,5,8
2,3,6,9


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
from IPython.display import display, HTML

# Function to save a styled DataFrame with a heatmap as a PNG
def save_styled_df_as_png(df, output_file):
    # Add background gradient (heatmap) to numeric columns
    styled_html = df.style.background_gradient(cmap="coolwarm").set_table_attributes(
        'style="border-collapse: collapse; width: auto; border: 1px solid black;"').to_html()

    # Save the HTML table to a temporary file
    html_file = os.path.abspath("temp_table.html")  # Get absolute path to avoid file resolution issues
    with open(html_file, "w") as f:
        f.write(f"""
        <html>
        <head>
        <style>
        body {{
            margin: 0;
            padding: 0;
            display: inline-block;
        }}
        table {{
            margin: 0 auto;  /* Center the table */
        }}
        </style>
        </head>
        <body>
        {styled_html}
        </body>
        </html>
        """)

    # Set up Selenium WebDriver in headless mode
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    # Automatically detect and download the correct ChromeDriver version
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Open the temporary HTML file
    driver.get(f"file://{html_file}")

    # Dynamically adjust window size to match the table size exactly
    table_width = driver.execute_script("return document.querySelector('table').offsetWidth")
    table_height = driver.execute_script("return document.querySelector('table').offsetHeight")
    driver.set_window_size(table_width, table_height)

    # Take a screenshot of the table
    driver.save_screenshot(output_file)
    driver.quit()

    print(f"PNG saved as '{output_file}' in the current directory.")

    # Display the styled table in the notebook
    display(HTML(styled_html))

# Example DataFrame
data = {
    'Column1': [1, 2, 3],
    'Column2': [4, 5, 6],
    'Column3': [7, 8, 9]
}
df = pd.DataFrame(data)

# Save the DataFrame as a PNG
output_file = "styled_table_with_heatmap.png"
save_styled_df_as_png(df, output_file)


PNG saved as 'styled_table_with_heatmap.png' in the current directory.


,Column1,Column2,Column3
0,1,4,7
1,2,5,8
2,3,6,9


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import pandas as pd
import os
from IPython.display import display, HTML

# Function to save a styled DataFrame with a heatmap as a PNG
def save_styled_df_as_png(df, output_file):
    # Add background gradient (heatmap) to numeric columns
    styled_html = df.style.background_gradient(cmap="coolwarm").set_table_attributes(
        'style="border-collapse: collapse; border: 1px solid black;"').to_html()

    # Save the HTML table to a temporary file
    html_file = os.path.abspath("temp_table.html")
    with open(html_file, "w") as f:
        f.write(f"""
        <html>
        <head>
        <style>
        body {{
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }}
        table {{
            border-collapse: collapse;
            margin: auto;
        }}
        </style>
        </head>
        <body>
        {styled_html}
        </body>
        </html>
        """)

    # Set up Selenium WebDriver in headless mode
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    # Automatically detect and download the correct ChromeDriver version
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Open the temporary HTML file
    driver.get(f"file://{html_file}")

    # Wait for the table to load and measure its size
    table = driver.find_element("tag name", "table")
    table_width = table.size["width"]
    table_height = table.size["height"]

    # Dynamically adjust window size to fit the table exactly
    driver.set_window_size(table_width + 50, table_height + 50)

    # Scroll to the table (in case it’s not fully visible)
    driver.execute_script("arguments[0].scrollIntoView(true);", table)

    # Take a full-page screenshot
    screenshot_path = "full_screenshot.png"
    driver.save_screenshot(screenshot_path)

    # Get table location and crop the screenshot to fit the table
    location = table.location
    x, y = int(location["x"]), int(location["y"])
    width, height = int(table_width), int(table_height)

    # Crop the image
    image = Image.open(screenshot_path)
    cropped_image = image.crop((x, y, x + width, y + height))
    cropped_image.save(output_file)

    # Clean up temporary files
    driver.quit()
    os.remove(screenshot_path)
    os.remove(html_file)

    print(f"PNG saved as '{output_file}' in the current directory.")

    # Display the styled table in the notebook
    display(HTML(styled_html))

# Example DataFrame
data = {
    'Column1': [1, 2, 3],
    'Column2': [4, 5, 6],
    'Column3': [7, 8, 9]
}
df = pd.DataFrame(data)

# Save the DataFrame as a PNG
output_file = "styled_table_with_heatmap.png"
save_styled_df_as_png(df, output_file)


PNG saved as 'styled_table_with_heatmap.png' in the current directory.


,Column1,Column2,Column3
0,1,4,7
1,2,5,8
2,3,6,9
